Rerun preprocessing that was done previously

In [2]:
import pandas as pd
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(link,header=0)[0]
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned', df.Neighbourhood)

Get location data using geocoder

In [12]:
df['latitude'] = 0
df['longitude'] = 0

In [13]:
import geocoder

In [14]:
def getData(x):
    
    postal_code = x.Postcode
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    x.latitude = lat_lng_coords[0]
    x.longitude = lat_lng_coords[1]
    return

In [ ]:
df['latitude']=df.apply(getData, axis=1)
df.head()

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


No success, read the data from given link

In [24]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')

In [20]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge dataframes

In [29]:
df_geo.rename(index=str, columns={"Postal Code":"Postcode"}, inplace=True)

In [30]:
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
df = df.merge(df_geo, how = 'inner', on = ['Postcode'])
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
